# Importation

In [1]:
#pip install pyspark (182MB)

# pip install pandas

In [295]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd
import re
from pyspark.sql.functions import *
import pyspark.sql.functions as f

In [ ]:
sc = SparkContext()

In [231]:
spark = SparkSession.builder.appName("Read CSV").getOrCreate()

# co2 = spark.read.csv("CO2/CO2.csv", header=True, inferSchema=True)
# Client1 = spark.read.csv("Client1/Clients_61.csv", header=True, inferSchema=True)

In [232]:
from pyhive import hive
conn = hive.connect(host='localhost', port=10000)
cursor = conn.cursor()



'''
the function takes a table name as argument take do a select to retrieve the data
tranform the result into Spark dataframe 
and transform the columns
'''
def import_data(table_name):
    cursor.execute(f"SELECT * FROM {table_name}")
    result = cursor.fetchall()
    data = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
    # Column transformation
    col = data.columns
    col = col.str.replace(f'{table_name}.', '')
    data.columns = col
    data = spark.createDataFrame(data)

    return data

## IMPORT CO2

In [286]:
co2 = import_data('co2_hive_ext')
co2_map = import_data('co2_hive_ext')

In [234]:
co2.show()

+---+--------------------+---------------+-------------+-----------+
| id|       marque_modele|    bonus_malus| rejetsco2gkm|coutenergie|
+---+--------------------+---------------+-------------+-----------+
|  2|AUDI E-TRON SPORT...|  -6Â 000â‚¬Â 1|            0|   319Â â‚¬|
|  3|AUDI E-TRON SPORT...|  -6Â 000â‚¬Â 1|            0|   356Â â‚¬|
|  4|AUDI E-TRON 55 (4...|  -6Â 000â‚¬Â 1|            0|   357Â â‚¬|
|  5|AUDI E-TRON 50 (3...|  -6Â 000â‚¬Â 1|            0|   356Â â‚¬|
|  6|       BMW i3 120 Ah|  -6Â 000â‚¬Â 1|            0|   204Â â‚¬|
|  7|      BMW i3s 120 Ah|  -6Â 000â‚¬Â 1|            0|   204Â â‚¬|
|  8|    CITROEN BERLINGO|  -6Â 000â‚¬Â 1|            0|   203Â â‚¬|
|  9|      CITROEN C-ZERO|  -6Â 000â‚¬Â 1|            0|   491Â â‚¬|
| 10|DS DS3 CROSSBACK ...|  -6Â 000â‚¬Â 1|            0|   251Â â‚¬|
| 11|HYUNDAI KONA elec...|  -6Â 000â‚¬Â 1|            0|   205Â â‚¬|
| 12|HYUNDAI KONA elec...|  -6Â 000â‚¬Â 1|            0|   205Â â‚¬|
| 13|JAGUAR I-PACE EV4...|  -6Â 00

In [235]:
co2 = co2.filter((f.col("bonus_malus") != " 150kW (204ch)\"") & (f.col("bonus_malus") != ' 100kW (136ch)"'))


In [236]:
co2.show()

+---+--------------------+-------------+------------+-----------+
| id|       marque_modele|  bonus_malus|rejetsco2gkm|coutenergie|
+---+--------------------+-------------+------------+-----------+
|  2|AUDI E-TRON SPORT...|-6Â 000â‚¬Â 1|           0|   319Â â‚¬|
|  3|AUDI E-TRON SPORT...|-6Â 000â‚¬Â 1|           0|   356Â â‚¬|
|  4|AUDI E-TRON 55 (4...|-6Â 000â‚¬Â 1|           0|   357Â â‚¬|
|  5|AUDI E-TRON 50 (3...|-6Â 000â‚¬Â 1|           0|   356Â â‚¬|
|  6|       BMW i3 120 Ah|-6Â 000â‚¬Â 1|           0|   204Â â‚¬|
|  7|      BMW i3s 120 Ah|-6Â 000â‚¬Â 1|           0|   204Â â‚¬|
|  8|    CITROEN BERLINGO|-6Â 000â‚¬Â 1|           0|   203Â â‚¬|
|  9|      CITROEN C-ZERO|-6Â 000â‚¬Â 1|           0|   491Â â‚¬|
| 10|DS DS3 CROSSBACK ...|-6Â 000â‚¬Â 1|           0|   251Â â‚¬|
| 11|HYUNDAI KONA elec...|-6Â 000â‚¬Â 1|           0|   205Â â‚¬|
| 12|HYUNDAI KONA elec...|-6Â 000â‚¬Â 1|           0|   205Â â‚¬|
| 13|JAGUAR I-PACE EV4...|-6Â 000â‚¬Â 1|           0|   271Â â‚¬|
| 16|KIA S

# 1- Division entre marque et modele

In [237]:
split_col = split(co2['marque_modele'], ' ')

In [238]:
marque = split_col.getItem(0)

In [239]:
co2 = co2.withColumn('marque', marque)

In [240]:
co2.select("marque").distinct().show()

+----------+
|    marque|
+----------+
|  MERCEDES|
|   PORSCHE|
|   HYUNDAI|
|    TOYOTA|
|     SKODA|
|    NISSAN|
|      LAND|
|   CITROEN|
|   BENTLEY|
|      AUDI|
|      MINI|
|   PEUGEOT|
|    JAGUAR|
|     VOLVO|
|     TESLA|
|       BMW|
|VOLKSWAGEN|
|       KIA|
|     SMART|
|   RENAULT|
+----------+
only showing top 20 rows



In [241]:
co2 = co2.withColumn("marque", regexp_replace("marque", "\"", ""))

In [242]:
co2.select("marque").distinct().show()

+----------+
|    marque|
+----------+
|  MERCEDES|
|   PORSCHE|
|   HYUNDAI|
|    TOYOTA|
|     SKODA|
|    NISSAN|
|      LAND|
|   CITROEN|
|   BENTLEY|
|      AUDI|
|      MINI|
|   PEUGEOT|
|    JAGUAR|
|     VOLVO|
|     TESLA|
|       BMW|
|VOLKSWAGEN|
|       KIA|
|     SMART|
|   RENAULT|
+----------+
only showing top 20 rows



In [243]:
co2 = co2.withColumn(
    "model",
    expr("substring(`marque_modele`, length(`Marque`) + 2)")
)

In [244]:
co2.select("model").show()

+--------------------+
|               model|
+--------------------+
|E-TRON SPORTBACK ...|
|E-TRON SPORTBACK ...|
|E-TRON 55 (408ch)...|
|E-TRON 50 (313ch)...|
|           i3 120 Ah|
|          i3s 120 Ah|
|            BERLINGO|
|              C-ZERO|
|DS3 CROSSBACK E-T...|
|KONA electric 64 kWh|
|KONA electric 39 kWh|
|I-PACE EV400 (400...|
|SOUL Moteur ÃƒÂ©l...|
|SOUL Moteur ÃƒÂ©l...|
|      EQC 400 4MATIC|
|VITO Tourer long ...|
|VITO Tourer extra...|
|MINI Cooper SE Ha...|
|Nouvelle NISSAN LEAF|
|Nouvelle NISSAN LEAF|
+--------------------+
only showing top 20 rows



In [245]:
co2 = co2.drop("marque_modele")

In [246]:
### co2.show()""

# Traitement des valeur erronées et manquante sur Bonus / Malus

In [247]:
# co2.filter(col("bonus_malus").isNull()).show()

In [248]:
co2.select('bonus_malus').distinct().show()

+-------------+
|  bonus_malus|
+-------------+
|   +7Â 613â‚¬|
|-6Â 000â‚¬Â 1|
|   +7Â 890â‚¬|
|   +7Â 340â‚¬|
|   +6Â 810â‚¬|
|            -|
|   +7Â 073â‚¬|
|   +8Â 460â‚¬|
|   +8Â 173â‚¬|
|   +8Â 753â‚¬|
+-------------+



In [249]:
co2.select('bonus_malus').distinct().show()

+-------------+
|  bonus_malus|
+-------------+
|   +7Â 613â‚¬|
|-6Â 000â‚¬Â 1|
|   +7Â 890â‚¬|
|   +7Â 340â‚¬|
|   +6Â 810â‚¬|
|            -|
|   +7Â 073â‚¬|
|   +8Â 460â‚¬|
|   +8Â 173â‚¬|
|   +8Â 753â‚¬|
+-------------+



In [250]:
co2 = co2.withColumn("bonus_malus", regexp_replace("bonus_malus", "[^\\d-]+", "").cast("int"))

In [251]:
co2.select('bonus_malus').distinct().show()

+-----------+
|bonus_malus|
+-----------+
|       7340|
|       7613|
|       6810|
|       7890|
|       null|
|       7073|
|     -60001|
|       8460|
|       8753|
|       8173|
+-----------+



In [252]:
co2 = co2.withColumn("bonus_malus", f.when(f.col("bonus_malus") == -60001, -6000).otherwise(f.col("bonus_malus")))


In [253]:
co2.show()

+---+-----------+------------+-----------+--------+--------------------+
| id|bonus_malus|rejetsco2gkm|coutenergie|  marque|               model|
+---+-----------+------------+-----------+--------+--------------------+
|  2|      -6000|           0|   319Â â‚¬|    AUDI|E-TRON SPORTBACK ...|
|  3|      -6000|           0|   356Â â‚¬|    AUDI|E-TRON SPORTBACK ...|
|  4|      -6000|           0|   357Â â‚¬|    AUDI|E-TRON 55 (408ch)...|
|  5|      -6000|           0|   356Â â‚¬|    AUDI|E-TRON 50 (313ch)...|
|  6|      -6000|           0|   204Â â‚¬|     BMW|           i3 120 Ah|
|  7|      -6000|           0|   204Â â‚¬|     BMW|          i3s 120 Ah|
|  8|      -6000|           0|   203Â â‚¬| CITROEN|            BERLINGO|
|  9|      -6000|           0|   491Â â‚¬| CITROEN|              C-ZERO|
| 10|      -6000|           0|   251Â â‚¬|      DS|DS3 CROSSBACK E-T...|
| 11|      -6000|           0|   205Â â‚¬| HYUNDAI|KONA electric 64 kWh|
| 12|      -6000|           0|   205Â â‚¬| HYUNDAI|

In [254]:
# Calculate the mode of the column
mode_value = co2.groupBy('bonus_malus').count().sort(f.col('count').desc()).select('bonus_malus').first()[0]

# Replace missing values with the mode
co2 = co2.fillna({'bonus_malus': mode_value})

In [255]:
mode_value

8753

In [256]:
co2.select('bonus_malus').distinct().show()

+-----------+
|bonus_malus|
+-----------+
|       7340|
|       7613|
|      -6000|
|       6810|
|       7890|
|       8753|
|       7073|
|       8460|
|       8173|
+-----------+



In [257]:
# #from pyspark.sql.functions import concat, format_string, col

# co2 = co2.withColumn("Bonus_Malus", format_string("%d€", co2["Bonus / Malus"]))
# co2 = co2.drop("Bonus / Malus")

# Traitement sur Cout Energie

In [258]:
co2.select('coutenergie').distinct().show()

+-----------+
|coutenergie|
+-----------+
|    67Â â‚¬|
|   247Â â‚¬|
|    66Â â‚¬|
|    85Â â‚¬|
|    47Â â‚¬|
|    43Â â‚¬|
|   221Â â‚¬|
|   204Â â‚¬|
|   214Â â‚¬|
|    74Â â‚¬|
|   185Â â‚¬|
|   199Â â‚¬|
|    77Â â‚¬|
|   491Â â‚¬|
|    78Â â‚¬|
|   206Â â‚¬|
|   725Â â‚¬|
|   271Â â‚¬|
|   177Â â‚¬|
|   357Â â‚¬|
+-----------+
only showing top 20 rows



In [259]:
co2.columns

['id', 'bonus_malus', 'rejetsco2gkm', 'coutenergie', 'marque', 'model']

In [260]:
co2 = co2.withColumn("coutenergie", regexp_replace("coutenergie", "[^\\d-]+", "").cast("int"))

In [261]:
co2.select('coutenergie').distinct().show()

+-----------+
|coutenergie|
+-----------+
|        251|
|         85|
|         53|
|         78|
|        155|
|        210|
|        271|
|        209|
|        319|
|        725|
|         47|
|        177|
|        185|
|        291|
|        206|
|        715|
|        205|
|         54|
|        491|
|        235|
+-----------+
only showing top 20 rows



In [262]:
co2.select('rejetsco2gkm').distinct().show()

+------------+
|rejetsco2gkm|
+------------+
|          29|
|          69|
|          42|
|          30|
|          85|
|          35|
|          52|
|           0|
|          47|
|         179|
|          43|
|          31|
|          70|
|          26|
|          46|
|          41|
|          55|
|          38|
|          40|
|          58|
+------------+
only showing top 20 rows



In [263]:
co2 = co2.withColumn("bonus_malus", f.when(f.col("bonus_malus") == -60001, -6000).otherwise(f.col("bonus_malus")))


In [264]:
co2 = co2.withColumn("bonus_malus", regexp_replace("bonus_malus", "[^\\d-]+", "").cast("int"))

In [265]:
co2.select('bonus_malus').distinct().show()

+-----------+
|bonus_malus|
+-----------+
|       7340|
|       7613|
|      -6000|
|       6810|
|       7890|
|       8753|
|       7073|
|       8460|
|       8173|
+-----------+



In [266]:
co2.select('coutenergie').distinct().show()

+-----------+
|coutenergie|
+-----------+
|        251|
|         85|
|         53|
|         78|
|        155|
|        210|
|        271|
|        209|
|        319|
|        725|
|         47|
|        177|
|        185|
|        291|
|        206|
|        715|
|        205|
|         54|
|        491|
|        235|
+-----------+
only showing top 20 rows



In [267]:
co2.select('marque').distinct().show()

+----------+
|    marque|
+----------+
|  MERCEDES|
|   PORSCHE|
|   HYUNDAI|
|    TOYOTA|
|     SKODA|
|    NISSAN|
|      LAND|
|   CITROEN|
|   BENTLEY|
|      AUDI|
|      MINI|
|   PEUGEOT|
|    JAGUAR|
|     VOLVO|
|     TESLA|
|       BMW|
|VOLKSWAGEN|
|       KIA|
|     SMART|
|   RENAULT|
+----------+
only showing top 20 rows



# Jointure

In [268]:
catalogue = import_data('catalogue_hive_ext')

In [269]:
catalogue.show()

+----------+---------------+---------+-----------+--------+--------+-------+--------+-----+-----+
|    marque|            nom|puissance|   longueur|nbplaces|nbportes|couleur|occasion| prix|   id|
+----------+---------------+---------+-----------+--------+--------+-------+--------+-----+-----+
|     Volvo|         S80 T6|      272|tr�s longue|       5|       5|  blanc|    true|35350|0E-18|
|Volkswagen| Touran 2.0 FSI|      150|     longue|       7|       5|  rouge|   false|27340|0E-18|
|Volkswagen|   Golf 2.0 FSI|      150|    moyenne|       5|       5|   bleu|   false|22900|0E-18|
|      Saab|       9.3 1.8T|      150|     longue|       5|       5|   bleu|    true|27020|0E-18|
|      Saab|       9.3 1.8T|      150|     longue|       5|       5|   bleu|   false|38600|0E-18|
|   Renault|    Laguna 2.0T|      170|     longue|       5|       5|  blanc|    true|19110|0E-18|
|   Renault|    Laguna 2.0T|      170|     longue|       5|       5|   noir|   false|27300|0E-18|
|   Renault|    Lagu

In [270]:
catalogue.select('marque').distinct().show(21)

+----------+
|    marque|
+----------+
|Volkswagen|
|   Peugeot|
|    Jaguar|
|    Lancia|
|      Mini|
|       Kia|
|     Volvo|
|      Saab|
|     Honda|
|      Audi|
|      Seat|
|  Mercedes|
|   Renault|
|       BMW|
|     Dacia|
|     Skoda|
|  Daihatsu|
|    Nissan|
|      Fiat|
|      Ford|
|   Hyunda�|
+----------+



In [271]:
catalogue = catalogue.withColumn("marque", f.when(f.col("marque") == 'Hyunda�', 'Hyundai').otherwise(f.col("marque")))


In [272]:
co2.select('marque').distinct().show(21)

+----------+
|    marque|
+----------+
|  MERCEDES|
|   PORSCHE|
|   HYUNDAI|
|    TOYOTA|
|     SKODA|
|    NISSAN|
|      LAND|
|   CITROEN|
|   BENTLEY|
|      AUDI|
|      MINI|
|   PEUGEOT|
|    JAGUAR|
|     VOLVO|
|     TESLA|
|       BMW|
|VOLKSWAGEN|
|       KIA|
|     SMART|
|   RENAULT|
|        DS|
+----------+
only showing top 21 rows



In [273]:
catalogue = catalogue.withColumn("marque", upper(catalogue["marque"]))

In [274]:
co2.select('marque').distinct().show(21)

+----------+
|    marque|
+----------+
|  MERCEDES|
|   PORSCHE|
|   HYUNDAI|
|    TOYOTA|
|     SKODA|
|    NISSAN|
|      LAND|
|   CITROEN|
|   BENTLEY|
|      AUDI|
|      MINI|
|   PEUGEOT|
|    JAGUAR|
|     VOLVO|
|     TESLA|
|       BMW|
|VOLKSWAGEN|
|       KIA|
|     SMART|
|   RENAULT|
|        DS|
+----------+
only showing top 21 rows



In [275]:
voiture = catalogue.join(co2, "marque", "left")

In [276]:
voiture.show()

+--------+----+---------+-----------+--------+--------+-------+--------+-----+-----+---+-----------+------------+-----------+--------------------+
|  marque| nom|puissance|   longueur|nbplaces|nbportes|couleur|occasion| prix|   id| id|bonus_malus|rejetsco2gkm|coutenergie|               model|
+--------+----+---------+-----------+--------+--------+-------+--------+-----+-----+---+-----------+------------+-----------+--------------------+
|MERCEDES|S500|      306|tr�s longue|       5|       5|   noir|    true|70910|0E-18|459|       8753|         262|       1051|SPRINTER Combi 31...|
|MERCEDES|S500|      306|tr�s longue|       5|       5|   noir|    true|70910|0E-18|458|       8753|         262|       1051|SPRINTER Combi 31...|
|MERCEDES|S500|      306|tr�s longue|       5|       5|   noir|    true|70910|0E-18|457|       8753|         260|       1041|SPRINTER Combi 31...|
|MERCEDES|S500|      306|tr�s longue|       5|       5|   noir|    true|70910|0E-18|456|       8753|         260|     

In [277]:
voiture.toPandas()

,marque,nom,puissance,longueur,nbplaces,nbportes,couleur,occasion,prix,id,id,bonus_malus,rejetsco2gkm,coutenergie,model
0,MERCEDES,S500,306,tr�s longue,5,5,noir,True,70910,0E-18,459.0,8753.0,262,1051.0,SPRINTER Combi 319 CDI (190ch) 4x4 PTAC 3500 k...
1,MERCEDES,S500,306,tr�s longue,5,5,noir,True,70910,0E-18,458.0,8753.0,262,1051.0,SPRINTER Combi 319 CDI (190ch) 4x4 PTAC 3500 k...
2,MERCEDES,S500,306,tr�s longue,5,5,noir,True,70910,0E-18,457.0,8753.0,260,1041.0,SPRINTER Combi 319 CDI (190ch) 4x4 PTAC 3500 k...
3,MERCEDES,S500,306,tr�s longue,5,5,noir,True,70910,0E-18,456.0,8753.0,260,1041.0,SPRINTER Combi 319 CDI (190ch) 4x4 PTAC 3500 k...
4,MERCEDES,S500,306,tr�s longue,5,5,noir,True,70910,0E-18,455.0,8753.0,255,1020.0,SPRINTER Combi 319 CDI (190ch) 4x2 PTAC 3500 k...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7645,LANCIA,Ypsilon 1.4 16V,90,courte,5,3,bleu,False,13500,0E-18,NaN,NaN,None,NaN,None
7646,LANCIA,Ypsilon 1.4 16V,90,courte,5,3,gris,False,13500,0E-18,NaN,NaN,None,NaN,None
7647,LANCIA,Ypsilon 1.4 16V,90,courte,5,3,rouge,True,9450,0E-18,NaN,NaN,None,NaN,None
7648,LANCIA,Ypsilon 1.4 16V,90,courte,5,3,bleu,True,9450,0E-18,NaN,NaN,None,NaN,None


## TRAITEMENT AVEC MAP REDUCE

In [296]:
def mapper(line):  
    line = tuple(line)

    # colonne marque
    marque =  line[1].split(" ")[0].replace("\"", "").capitalize() #replace quote near Volswagen
    # colonne Malus/Bonus
    if line[2] in [" 150kW (204ch)\"", " 100kW (136ch)\""]:
        return (None, None)
#     malus_bonus = line[2].strip().replace("¬Â\xa01", "").replace(r"[\d-]+", "").replace("€", "").replace("\"", "")
    malus_bonus = line[2].strip().replace("¬Â\xa01", "").replace(r"[\d-]+", "").replace("€", "").replace("\"", "")

    malus_bonus = ''.join(re.findall(r"[\d-]+", malus_bonus))
    
#     missing values bonus_malus
    malus_bonus = "0" if len(malus_bonus) == 1 else malus_bonus
    # colonne cout energie
    cout = line[-1]
    cout_splitted = cout.replace('Â', '').split()
    cout = cout_splitted[0] + cout_splitted[1] if len(cout_splitted) == 3 else cout_splitted[0] if len(cout_splitted) == 2 else cout
    cout = cout.encode('ascii', 'ignore').decode('ascii').replace(',', '')
    
    # colonne Rejet CO2
    rejet = line[3]

    malus_bonusInt = int(malus_bonus)
    rejetInt = (rejet)
    coutInt = int(cout)

    # on crée le couple key value avec la marque comme key
    new_value = f"{malus_bonusInt}|{rejetInt}|{coutInt}"
    return (marque, new_value)

In [297]:
co2_mapped = co2_map.rdd.map(mapper)

In [298]:
co2_mapped.collect()

[('Audi', '-6000|0|319'),
 ('Audi', '-6000|0|356'),
 ('Audi', '-6000|0|357'),
 ('Audi', '-6000|0|356'),
 ('Bmw', '-6000|0|204'),
 ('Bmw', '-6000|0|204'),
 ('Citroen', '-6000|0|203'),
 ('Citroen', '-6000|0|491'),
 ('Ds', '-6000|0|251'),
 ('Hyundai', '-6000|0|205'),
 ('Hyundai', '-6000|0|205'),
 ('Jaguar', '-6000|0|271'),
 (None, None),
 (None, None),
 ('Kia', '-6000|0|214'),
 ('Kia', '-6000|0|214'),
 ('Mercedes', '-6000|0|291'),
 ('Mercedes', '-6000|0|411'),
 ('Mercedes', '-6000|0|411'),
 ('Mini', '-6000|0|199'),
 ('Nissan', '-6000|0|155'),
 ('Nissan', '-6000|0|177'),
 ('Peugeot', '-6000|0|221'),
 ('Peugeot', '-6000|0|241'),
 ('Peugeot', '-6000|0|203'),
 ('Renault', '-6000|0|206'),
 ('Renault', '-6000|0|206'),
 ('Renault', '-6000|0|206'),
 ('Skoda', '-6000|0|210'),
 ('Smart', '-6000|0|175'),
 ('Smart', '-6000|0|175'),
 ('Smart', '-6000|0|213'),
 ('Smart', '-6000|0|213'),
 ('Smart', '-6000|0|175'),
 ('Smart', '-6000|0|201'),
 ('Smart', '-6000|0|201'),
 ('Smart', '-6000|0|175'),
 ('Smart'